## Parse RSS

In [509]:
# %%writefile get_events.py
import feedparser
# from html2text import html2text
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

def get_rss_feed(URL):
    """Get RSS feed."""
    feed = feedparser.parse(URL)
    return feed if feed.entries else []

def parse_rss_entries(feed) -> list:
    """Parse XML in RSS feed."""
    records = []
    for each in feed.entries:
        title = each.title
        start_dt = each.get("bc_start_date_local")
        end_dt = each.get("bc_end_date_local")
        # address = f"{each.get('bc_number')} {each.get('bc_street')}, {each.get('bc_city')}, {each.get('bc_zip')}, {each.get('bc_state')}"
        location = each.get('bc_name')
        street = f"{each.get('bc_number')} {each.get('bc_street')}"
        city = each.get('bc_city')
        zip_code = each.get('bc_zip')
        tags = [f.get('term') for f in each.tags]
        description = each.description
        link = each.link
        is_virtual = each.get('bc_is_virtual')
        is_full = each.get('bc_is_full')
        is_require_signup = each.get('bc_is_required')
        loc_coord = [each.get('bc_latitude'), each.get('bc_longitude')]
        record = {'Title': title,
                  'Start Time': start_dt,
                  'End Time': end_dt,
                  'Location': location,
                  'Description': description,
                  'Event Page': link,
                  'Street': street,
                  'City': city,
                  'Zip Code': zip_code,
                  'Full': is_full,
                  'Require Signup': is_require_signup,
                  'Virtual': is_virtual,
                  'Categories': tags}
        records.append(record)
    return records

def clean_df(df):
    """Clean df values."""
    df['Description'] = (df['Description']
                         .apply(lambda x: re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', ' ', x))
                         .apply(lambda x: re.sub('[\n]+', ' ', x))
                         .apply(lambda x: re.sub('\s{2,}', ' ', x).strip())
                         )
    return df

def scrape_rss_feeds(libraries, end_date) -> pd.DataFrame:
    """Scrape library RSS feeds."""
    full_records = []
    for library in libraries:
        page = 1
        while True: 
            URL = f'https://gateway.bibliocommons.com/v2/libraries/{library}/rss/events?endDate={end_date}&page={page}'
            page_feed = get_rss_feed(URL)
            if not page_feed:
                print(f'No more events in page {page}. Break.')
                break
            records = parse_rss_entries(page_feed)
            full_records += records
            page += 1

    df = pd.DataFrame(full_records)
    df = clean_df(df)
    return df

In [506]:
CSV_PATH = '/Users/yuanhunglo/Downloads/events-v2.csv'

libraries = ['marinlibrary', 'sjpl', 'paloalto', 'sccl', 'smcl', 'ccclib', 'aclibrary']
end_date = (datetime.today() + timedelta(days=30)).strftime('%Y-%m-%d')
df = scrape_rss_feeds(libraries[:], end_date=end_date)
df.to_csv(CSV_PATH)

No more events in page 10. Break.
No more events in page 62. Break.
No more events in page 5. Break.
No more events in page 21. Break.
No more events in page 40. Break.
No more events in page 30. Break.
No more events in page 15. Break.


Approximately scrapes 10 entries per second.

## Google Sheet

In [507]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd

CSV_PATH = '/Users/yuanhunglo/Downloads/events-v2.csv'

gc = gspread.service_account(filename='/Users/yuanhunglo/.config/gspread/bay-area-library-events-search-811371cbbe60.json')

sh = gc.open_by_key('1l9tGXmmCu0IW5gqKG6jkpicyrrdSHNAQOKMBrCPe8UM')
# sh = gc.open("Bay Area Library Events Search")

worksheet = sh.worksheet("Sheet1")
# Clear sheet
worksheet.clear()

df = pd.read_csv(CSV_PATH, index_col=0)
# Write df to sheet
set_with_dataframe(worksheet, df)
# worksheet.update(range_name=[df.columns.values.tolist()], values=df.values.tolist())


## Streamlit

In [508]:
%%writefile /Users/yuanhunglo/Downloads/streamlit_app.py
import streamlit as st
from streamlit_gsheets import GSheetsConnection
from datetime import datetime
import pandas as pd
from ast import literal_eval
from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_any_dtype,
    is_numeric_dtype,
    is_object_dtype,
)

st.set_page_config(layout="wide")

# Create a connection object.
conn = st.connection("gsheets", type=GSheetsConnection)

df = conn.read()
# df = conn.read(
#     worksheet="Sheet1",
#     ttl="10m",
#     usecols=[0, 1],
#     nrows=3,
# )

# Preprocss df before 
def clean_df(df):
    # df['Tags'] = df['Tags'].apply(lambda x: [] if x == '' else literal_eval(x))
    df['Categories'] = df['Categories'].apply(pd.eval)
    # df['Loc Coord'] = df['Loc Coord'].apply(lambda x: eval(x))
    df['Zip Code'] = df['Zip Code'].astype('Int64').astype('str')
    # df = df.drop(columns=['Loc Coord'])
    return df

df = clean_df(df)

st.title("📚 Bay Area Library Events")

st.write(
    """Browse free public library events around the Bay Area in an easily searchable table. Use `Filters` to narrow down the list.
    The table refreshes every day at midnight.
    """
)


def filter_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds a UI on top of a dataframe to let viewers filter columns

    Args:
        df (pd.DataFrame): Original dataframe

    Returns:
        pd.DataFrame: Filtered dataframe
    """
    # modify = st.checkbox("`Filters`")

    # if not modify:
    #     return df

    df = df.copy()

    # Try to convert datetimes into a standard format (datetime, no timezone)
    for col in df.columns:
        if is_object_dtype(df[col]):
            try:
                df[col] = pd.to_datetime(df[col])
            except Exception:
                pass

        if is_datetime64_any_dtype(df[col]):
            df[col] = df[col].dt.tz_localize(None)

    modification_container = st.container()

    with modification_container:
        to_filter_columns = st.multiselect("Filter table on", df.columns, default=['Description', 'City', 'Start Time'])
        for column in to_filter_columns:
            left, right = st.columns((1, 20))
            left.write("↳")
            # Treat columns with < 10 unique values as categorical
            if column in ['City'] or is_categorical_dtype(df[column]) or df[column].nunique() < 10:
                user_cat_input = right.multiselect(
                    f"Values for {column}",
                    df[column].unique(),
                    # default=list(df[column].unique()),
                    default=[],
                )
                df = df[df[column].isin(user_cat_input)]
            elif is_numeric_dtype(df[column]):
                _min = float(df[column].min())
                _max = float(df[column].max())
                step = (_max - _min) / 100
                user_num_input = right.slider(
                    f"Values for {column}",
                    _min,
                    _max,
                    (_min, _max),
                    step=step,
                )
                df = df[df[column].between(*user_num_input)]
            elif is_datetime64_any_dtype(df[column]):
                user_date_input = right.date_input(
                    f"Values for {column}",
                    value=(
                        df[column].min(),
                        df[column].max(),
                    ),
                )
                if len(user_date_input) == 2:
                    user_date_input = tuple(map(pd.to_datetime, user_date_input))
                    start_date, end_date = user_date_input
                    df = df.loc[df[column].between(start_date, end_date)]
            else:
                user_text_input = right.text_input(
                    f'Search substring or regex in column "{column}"',
                )
                if user_text_input:
                    df = df[df[column].str.contains(user_text_input, case=False, regex=True)]

    return df


st.dataframe(
    filter_dataframe(df),
    column_config={
        "Start Time": st.column_config.DatetimeColumn(
            None,
            min_value=datetime(2023, 6, 1),
            max_value=datetime(2025, 1, 1),
            # format="MMM D, YYYY, h:mm a",
            # format="YYYY-MM-DD, h:mm a",
            format="M/D/YY, h:mm a",
            step=60
        ),     
        "End Time": st.column_config.DatetimeColumn(
            None,
            min_value=datetime(2023, 6, 1),
            max_value=datetime(2025, 1, 1),
            # format="MMM D, YYYY, h:mm a",
            # format="YYYY-MM-DD, h:mm a",
            format="M/D/YY, h:mm a",
            step=60
        ),     
        "Event Page": st.column_config.LinkColumn(
            None, display_text="Event page"
        ),       
        "Categories": st.column_config.ListColumn(
            None,
            help="The sales volume in the last 6 months",
            width="large",
        ),
    }
)
# st.dataframe(df)

# Print results.

# for row in df.itertuples():
#     st.write(f"__{row.title}__:")
#     st.write(f"{row.description}")

Overwriting /Users/yuanhunglo/Downloads/streamlit_app.py


## Use selenium to scrape library websites

In [ ]:
%%writefile fetch_data.py
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import os
import re
import json
import selenium

# Initialize Chrome Selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-extensions')
options.add_argument('--disable-gpu')
# user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537'
options.add_argument('user-agent={0}'.format(user_agent))

driver = webdriver.Chrome(options=options) 
# driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options) 
# driver.implicitly_wait(90)
driver.set_page_load_timeout(90)

# Load the URL and get the page source
URL = 'https://liveat678bellflower.com/floor-plans.aspx'
driver.get(URL)

results_container = WebDriverWait(driver, 100).until(
    EC.presence_of_element_located((By.ID, 'par_10617247'))
)

assert type(results_container) == selenium.webdriver.remote.webelement.WebElement, "results_container is a wrong type! Please check."

sections = results_container.find_elements(By.CLASS_NAME, "unit-container")

# Find elements
res = []
for idx, section in enumerate(sections):
    unit_type_str = section.find_element(By.XPATH, '//*[@id="text-area"]/div/div[1]/h2').get_attribute("innerHTML").strip()
    unit_price_str = section.find_element(By.CLASS_NAME, "unit-rent").get_attribute("innerHTML")
    unit_number_str = section.find_element(By.CLASS_NAME, "unit-number").get_attribute("innerHTML")
    unit_sqft_str = section.find_element(By.CLASS_NAME, "unit-sqft").get_attribute("innerHTML")

    unit_sqft = int(re.findall(r'\d+', unit_sqft_str.replace(',', ''))[0])
    unit_number = int(re.findall(r'\d+', unit_number_str)[0])
    unit_price = unit_price_str.replace('$', '').replace(',', '')
    unit_price = int(unit_price[-4:])
    
    assert unit_type_str == 'The Preserve', 'Wrong unit type! Please check.'
    
    _res = {"unit_number": unit_number,
            "unit_price": unit_price,
            "unit_sqft": unit_sqft
            }
    
    res.append(_res)

assert len(res) != 0, "Result list is empty! Please check."

res2 = sorted(res, key=lambda x: (x['unit_price'], x['unit_number']))

# Write out results
with open("unit_prices.json", "w") as f:
    # Dump the list to the file
    json.dump(res2, f)


Overwriting fetch_data.py


In [5]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import os
import re
import json

# Initialize Chrome Selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-extensions')
options.add_argument('--disable-gpu')
# user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537'
options.add_argument('user-agent={0}'.format(user_agent))
options.add_experimental_option(
    "prefs", {
        # block image loading
        "profile.managed_default_content_settings.images": 2,
    }
)

driver = webdriver.Chrome(options=options) 
# driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options) 
# driver.set_page_load_timeout(90)
driver.implicitly_wait(90)

In [6]:
URL = 'https://sjpl.bibliocommons.com/'

# Load the URL and get the page source
driver.get(URL)

In [9]:
results_container = WebDriverWait(driver, 100).until(
    # EC.presence_of_element_located((By.CLASS_NAME, "result-items"))
    EC.presence_of_element_located((By.CSS_SELECTOR, "#content > div > div.cp-events-search-page > div > div > div.row > div.col-lg-9.col-xl-9 > div > ul"))
)

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001009d853c chromedriver + 3966268
1   chromedriver                        0x00000001009d0ac8 chromedriver + 3934920
2   chromedriver                        0x0000000100653da0 chromedriver + 277920
3   chromedriver                        0x0000000100696394 chromedriver + 549780
4   chromedriver                        0x00000001006cebf0 chromedriver + 781296
5   chromedriver                        0x000000010068afb0 chromedriver + 503728
6   chromedriver                        0x000000010068ba28 chromedriver + 506408
7   chromedriver                        0x000000010099d724 chromedriver + 3725092
8   chromedriver                        0x00000001009a1c18 chromedriver + 3742744
9   chromedriver                        0x000000010098620c chromedriver + 3629580
10  chromedriver                        0x00000001009a2714 chromedriver + 3745556
11  chromedriver                        0x0000000100979584 chro

In [ ]:
sections = results_container.find_elements(By.CLASS_NAME, "event-details")

In [39]:
# section.find_element(By.CSS_SELECTOR, "div.unit-container-left > div:nth-child(3)").get_attribute("innerHTML")
# section.find_element(By.CSS_SELECTOR, "div.unit-container-left > div:nth-child(4)").get_attribute("innerHTML").replace('Lease Term: ', '')

'12'

## Use bs4 to scrape websites

In [186]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import time

def make_url(base_url, params):
    """Make URL."""
    pass


def get_html(URL):
    """Get page HTML."""
    html_content = requests.get(url=URL).text
    soup = BeautifulSoup(html_content, 'html.parser')
    tags = soup.find_all(class_='cp-events-search-item')
    return tags


def get_info(tag):
    """Get info from HTML."""
    # print('\n')
    link = tag.find('a', class_='cp-link')
    event_loc = tag.find('div', class_='cp-event-location')#
    dt_ele = tag.find('div', class_='cp-event-date-time').find_all('span', class_='cp-screen-reader-message')
    date = dt_ele[0].text
    event_time = 'all day' if len(dt_ele) == 1 else dt_ele[1].text
    # date_time = tag.find('div', class_='cp-event-date-time').find_all('span', class_='cp-screen-reader-message').text
    link_text = re.sub(r'Featured Event.*', '', link.text)
    # print(link_text, '\t', date)
    
    info = [
        link_text, # link text
        link.get('href'), # link URL
        tag.find('p').text, # description
        # re.findall(r'\w+\s\d{1,2}, \d{4}', date)[0], # date
        date.replace('on ', ''),
        event_time, # time
        event_loc.find('span').text, # location
        event_loc.find('a').get('href') if event_loc.find('a') else '', # location link
        [r.find('span').text for r in tag.find('div', class_='cp-event-taxonomies').find_all('li')], # tags
    ]
    return info

def get_full_description():
    # TODO: go into event page to get full event description.
    pass


In [177]:

# tag.find('a', class_='cp-link').find('span')
# tag.find('div', class_='cp-event-date-time').find('span', class_='cp-screen-reader-message')
# tag.find('div', class_='cp-event-date-time').find('span', class_='cp-screen-reader-message').text
feed = tag.find('div', class_='cp-event-date-time').find_all('span', class_='cp-screen-reader-message')
feed[0].text
# r[1].text


'on March 10, 2024'

In [218]:
(datetime.today() + timedelta(days=7)).strftime('%Y-%m-%d')

'2024-03-11'

In [219]:

df = []
# library = 'smcl' # 
end_date = (datetime.today() + timedelta(days=7)).strftime('%Y-%m-%d')

# Get page
for library in ['paloalto', 'sjpl', 'sccl', 'smcl']:
# for library in ['paloalto']:
    page = 1
    while True:
        URL = f'https://{library}.bibliocommons.com/v2/events?endDate={end_date}&page={page}'
        print(URL)
        tags = get_html(URL)
        if not tags: 
            print(f'No more events in page {page}. Break.')
            break
        
        # Scrape
        for tag in tags:
            info = get_info(tag)
            df.append(info)
        page += 1


https://paloalto.bibliocommons.com/v2/events?endDate=2024-03-11&page=1
https://paloalto.bibliocommons.com/v2/events?endDate=2024-03-11&page=2
No more events after page 2. Break.
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=1
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=2
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=3
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=4
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=5
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=6
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=7
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=8
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=9
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=10
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=11
https://sjpl.bibliocommons.com/v2/events?endDate=2024-03-11&page=12
https://sjpl.bi

In [233]:
df2 = pd.DataFrame(df, columns=['event_name', 'event_url', 'description', 'event_date', 'event_time', 'location', 'location_url', 'tags'])
# df2.sample(20)


In [ ]:
# Keyword search the description
df2.query("description.str.contains(r'(story|math)')").description.values

/var/folders/zq/55w3ggnn14n4zkb7cv5vvnb80000gn/T/ipykernel_18366/257470363.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df2.query("description.str.contains(r'(story|math)')").description.values


array(["Join Librarian Melisa on Tuesdays at 11:00 AM for Family Storytime at Children's Library. Come sing, rhyme, and read stories with us! While this storytime will be geared towards those older than 2 years, all ages are welcome!",
       'Join Librarian Devon on Wednesday evenings at 7:00pm for Night Owls at the Mitchell Park Library.\xa0Feel free to come decked out in your coziest pajamas to sing, rhyme, and read stories with us!\xa0This storytime is for kids of all…',
       'Filmed in Canada, Iran, and the United States, Secrets of the Surface: The Mathematical Vision of Maryam Mirzakhani examines the life and mathematical work of Maryam Mirzakhani, an Iranian immigrant to the United States who became a…',
       'Join Librarian Devon on Fridays at 11:00 AM for Family Storytime at Downtown Library. Come sing, rhyme, and read stories with us! While this storytime will be geared towards those older than 2 years, all ages are welcome!',
       'Join us\xa0for Indoor Music and Move

## Discord bot

In [4]:
import shutil
shutil.copyfile('latest_prices.json', 'snapshot_prices.json')

'snapshot_prices.json'

In [2]:
# %%writefile discord_bot.py
import discord

bot = discord.Client()

# Event triggered when the bot is ready
@bot.event
async def on_ready():
    print(f'Logged in as {bot.user.name} ({bot.user.id})')
    channel = bot.get_channel(1084864988688154627) 
    await channel.send("{'unit_number': 2, 'unit_price': 3403, 'unit_sqft': 1076, 'available_date': '04/06/2024'}")


bot.run(os.getenv('DISCORD_TOKEN'))


RuntimeError: Cannot close a running event loop

In [ ]:
# %%writefile discord_bot.py
import discord

bot = discord.Client()

# Event triggered when the bot is ready
@bot.event
async def on_ready():
    print(f'Logged in as {bot.user.name} ({bot.user.id})')
    channel = bot.get_channel(1084864988688154627) 
    await channel.send("{'unit_number': 2, 'unit_price': 3403, 'unit_sqft': 1076, 'available_date': '04/06/2024'}")


bot.run(os.getenv('DISCORD_TOKEN'))


RuntimeError: Cannot close a running event loop